In [3]:
import quandl
import pandas as pd
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup as bs
import pickle
from datetime import datetime, timedelta, date
%matplotlib notebook

# Quandl
https://www.quandl.com/tools/python

In [11]:
# 1) Quandl
# https://www.quandl.com/data/BCRA/MVAR_TIPCAMBREF-Reference-Exchange-Rate-AR-against-US
tickers = ['BCRA/MVAR_TIPCAMBREF']
df_exchange = quandl.get(tickers, start_date="2014-1-01")

In [12]:
df_exchange.head(5)

,BCRA/MVAR_TIPCAMBREF - Amount
Date,
2014-01-02,6.5430
2014-01-03,6.5580
2014-01-06,6.5857
2014-01-07,6.5988
2014-01-08,6.6042


In [13]:
# 2) Quandl
# https://www.quandl.com/search?filters=%5B%22Free%22%5D&query=cme%20cl
# https://www.quandl.com/data/CME/CLM2017-Crude-Oil-Futures-June-2017-CLM2017
tickers = ['CME/CLM2017']
df_CLM7 = quandl.get(tickers, start_date="2014-1-01")

In [14]:
df_CLM7

,CME/CLM2017 - Open,CME/CLM2017 - High,CME/CLM2017 - Low,CME/CLM2017 - Last,CME/CLM2017 - Change,CME/CLM2017 - Settle,CME/CLM2017 - Volume,CME/CLM2017 - Previous Day Open Interest
Date,,,,,,,,
2014-01-02,0.00,0.00,0.00,NaN,NaN,79.76,1000.0,4809.0
2014-01-03,0.00,0.00,0.00,NaN,NaN,79.38,100.0,4809.0
2014-01-06,0.00,0.00,0.00,NaN,NaN,79.52,100.0,4809.0
2014-01-07,0.00,0.00,0.00,NaN,NaN,78.45,100.0,4809.0
2014-01-08,0.00,0.00,0.00,NaN,NaN,77.82,101.0,4809.0
2014-01-09,0.00,0.00,0.00,NaN,NaN,77.95,101.0,4809.0
2014-01-10,0.00,0.00,77.73,NaN,NaN,77.96,101.0,4809.0
2014-01-13,0.00,0.00,77.70,NaN,NaN,77.78,101.0,4809.0
2014-01-14,78.05,78.05,78.05,NaN,NaN,78.46,115.0,4809.0


In [ ]:
# 3) Quandl use API KEY

# To get the API KEY: go to quandl.com, log in, and then go to https://www.quandl.com/account/api .
tickers = ['CME/CLG2017']
quandl.ApiConfig.api_key = *********
df_CLM7 = quandl.get(tickers, start_date="2014-1-01")

# Yahoo Finance
example:
https://finance.yahoo.com/quote/aapl/history/

We can scrape Yahoo Finance

In [4]:
def get_historical_data(name, starting_date):
    today = datetime(datetime.today().year, datetime.today().month, datetime.today().day)
    data = []
    first_date = starting_date
    second_date = min(starting_date + timedelta(days=99), today)
    while(first_date<second_date):
        p1 = str(int(first_date.timestamp()))
        p2 = str(int(second_date.timestamp()))
        url = "https://finance.yahoo.com/quote/" + name + "/history?period1="+ p1 + "&period2="+ p2 +\
        "&interval=1d&filter=history&frequency=1d"
        rows = bs(urllib.request.urlopen(url).read(), "lxml").findAll('table')[0].tbody.findAll('tr')

        for each_row in reversed(rows):
            divs = each_row.findAll('td')
            if divs[1].span.text != 'Dividend' or divs[1].span.text  != 'Dividend': #Ignore this row in the table
                #I'm only interested in 'Open' price; For other values, play with divs[1 - 5]
                try:
                    data.append({
                        'Date': divs[0].span.text, 'Open': float(divs[1].span.text.replace(',','')),
                        'High': float(divs[2].span.text.replace(',','')),
                        'Low': float(divs[3].span.text.replace(',','')),
                        'Close': float(divs[4].span.text.replace(',','')),
                        'Adj Close': float(divs[5].span.text.replace(',','')),
                        'Volume': float(divs[6].span.text.replace(',',''))
                    })
                except:
                    print(divs)
                    pass
        first_date = min(first_date + timedelta(days=99), today)
        second_date = min(second_date + timedelta(days=99), today)

    return pd.DataFrame(data=data).set_index('Date')

In [5]:
df_AAPL = get_historical_data('AAPL', datetime(2010, 11, 21))

[<td class="Py(10px) Ta(start) Pend(10px)" data-reactid="1002"><span data-reactid="1003">Jun 09, 2014</span></td>, <td class="Ta(c) Py(10px) Pstart(10px)" colspan="6" data-reactid="1004"><strong data-reactid="1005">7/1</strong><!-- react-text: 1006 --> <!-- /react-text --><span data-reactid="1007">Stock Split</span></td>]


In [6]:
df_AAPL

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
"Nov 22, 2010",30.19,44.77,44.77,43.70,43.81,98268800.0
"Nov 23, 2010",29.74,44.10,44.54,43.79,44.35,129861900.0
"Nov 24, 2010",30.33,44.97,45.06,44.54,44.57,103431300.0
"Nov 26, 2010",30.35,45.00,45.39,44.71,44.82,59396400.0
"Nov 29, 2010",30.53,45.27,45.35,44.48,45.07,111446300.0
"Nov 30, 2010",29.98,44.45,44.91,44.41,44.79,125464500.0
"Dec 01, 2010",30.48,45.20,45.39,45.00,45.04,115437700.0
"Dec 02, 2010",30.65,45.45,45.57,44.98,45.36,115709300.0
"Dec 03, 2010",30.58,45.35,45.52,45.19,45.29,85523200.0
